In [ ]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
gpus=tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)
gpus

In [ ]:
import cv2
import imghdr

In [ ]:
data_dir='/kaggle/input/dataset/Train/Train'

In [ ]:
arr=cv2.imread(os.path.join(data_dir,'Anthracnose','20211008_124249 (Custom).jpg'))
arr.shape

In [ ]:
plt.imshow(cv2.cvtColor(arr,cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
data=tf.keras.utils.image_dataset_from_directory('/kaggle/input/dataset/Train/Train',batch_size=128,image_size=(320,240))
data1=tf.keras.utils.image_dataset_from_directory('/kaggle/input/dataset/Test/Test',batch_size=128,image_size=(320,240))

In [ ]:
iterator=data.as_numpy_iterator()
iterator1=data1.as_numpy_iterator()

In [ ]:
batch=iterator.next()
batch1=iterator1.next()
print(batch[0].shape)
print(batch1[0].shape)

In [ ]:
batch[1]

In [ ]:
fig,ax=plt.subplots(ncols=4,figsize=(20,20))
for idx,img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
scaling=batch[0]/255

In [ ]:
scaling.min()

In [ ]:
data =data.map(lambda x,y:(x/255,y))
data1=data1.map(lambda x,y:(x/255,y))

In [ ]:
batch=data.as_numpy_iterator().next()
batch[0].max()

In [ ]:
batch1=data1.as_numpy_iterator().next()
batch1[0].max()

In [ ]:
batch

In [ ]:
batch1

In [ ]:
fig,ax=plt.subplots(ncols=4, figsize=(20,20))
for idx,img in enumerate(batch[0][:4]):
    ax[idx].imshow(img)
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
print(len(data))
len(data1)

In [ ]:
train=data
test=data1

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense 

In [ ]:
model=Sequential()

In [ ]:
model.add(Conv2D(16,(3,3),1,activation='relu',input_shape=(320,240,3)))
model.add(MaxPooling2D())

model.add(Conv2D(32,(3,3),1,activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16,(3,3),1,activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(1024,activation='relu'))
model.add(Dense(8,activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
os.mkdir('logs')

In [ ]:
logdir='/kaggle/working/logs'

In [ ]:
tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist=model.fit(train,epochs=65,callbacks=[tensorboard_callback])

In [ ]:
hist.history

In [ ]:
model = tf.keras.models.load_model('/kaggle/input/models/model7.h5')

In [ ]:
fig,ax=plt.subplots(figsize=(5,5))
plt.plot(hist.history['accuracy'],color='teal',label='Accuracy')
plt.plot(hist.history['loss'],color='r',label='Loss')
plt.suptitle('Loss and Accuracy')
plt.legend(loc='upper left')
plt.show()

In [ ]:
from tensorflow.keras.metrics import SparseCategoricalAccuracy

In [ ]:
spa=SparseCategoricalAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    x,y=batch
    yhat=model.predict(x)

    spa.update_state(y,yhat)


In [ ]:
print(f'Accuracy {spa.result().numpy()}')